In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv('bank_marketing_part1_Data-1.csv')

In [ ]:
df.head(1)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
ax = sns.boxplot(data=df,width=0.6,palette="Set3",orient='h',linewidth=0.5)

In [ ]:
fig,ax = plt.subplots(figsize=(5, 2.5))   
sns.heatmap(df.corr(), ax=ax, annot=True, linewidths=0.05, fmt= '.2f',cmap="magma") # the color intensity is based on 
plt.show()

In [ ]:
from scipy import stats
z = np.abs(stats.zscore(df))   # get the z-score of every value with respect to their columns
print(z)

In [ ]:
threshold = 3
np.where(z > threshold)

In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
np.where((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR)))

# Select only the rows without a single outlier
df1 = df[(z < 3).all(axis=1)]    
df1.shape, df.shape

In [ ]:
#make a deep copy of the dataframe
df2 = df.copy()  

#Replace all the outliers with median values. This will create new some outliers but, we will ignore them

# iterate using 2 variables.i for rows and j for columns
for i, j in zip(np.where(z > threshold)[0], np.where(z > threshold)[1]):
    # replace i,jth element with the median of j i.e, corresponding column
    df2.iloc[i,j] = df.iloc[:,j].median()  

# New outliers detected after imputing the original outliers 
z = np.abs(stats.zscore(df2))
np.where(z > threshold)  

df1.shape, df2.shape

In [ ]:
ax = sns.boxplot(data=df1,width=0.6,palette="Set3",orient='h',linewidth=0.5)

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
scaled_df1 = pd.DataFrame(scaler.fit_transform(df1.iloc[:,1:7]),columns=df1.columns[1:])
scaled_df1.head()

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
wardlink = linkage(scaled_df1, method = 'ward')
dend = dendrogram(wardlink)

In [ ]:
dend = dendrogram(wardlink,
                 truncate_mode='lastp',
                 p = 210,
                 )

In [ ]:
from scipy.cluster.hierarchy import fcluster
clusters = fcluster(wardlink, 23, criterion='distance')
clusters

In [ ]:
scaled_df1['clusters'] = clusters
scaled_df1.head()

In [ ]:
#scaled_df1.to_csv('hc.csv')

#### K-means

In [ ]:
from sklearn.cluster import KMeans 
k_means = KMeans(n_clusters = 3)
k_means.fit(scaled_df1)

In [ ]:
k_means.labels_

In [ ]:
k_means.inertia_

In [ ]:
k_means = KMeans(n_clusters = 1)
k_means.fit(scaled_df1)
k_means.inertia_

In [ ]:
wss =[] 

In [ ]:
for i in range(1,11):
    KM = KMeans(n_clusters=i)
    KM.fit(scaled_df1)
    wss.append(KM.inertia_)

In [ ]:
wss

In [ ]:
plt.plot(range(1,11), wss)

In [ ]:
k_means = KMeans(n_clusters = 3)
k_means.fit(scaled_df1)
labels = k_means.labels_
#
df1["Clus_kmeans"] = labels
df1.head(2)

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
silhouette_score(scaled_df1,labels)

In [ ]:
silhouette_samples(scaled_df1,labels).min()